In [1]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator
from langchain_core.messages import ToolCall, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
 
# Define AgentState
class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
 
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv()
 
# Import tools
from utils.tavily.core import web_search
from utils.pytract.core import pytract_rag
 
# Tool definitions
@tool("web_search")
def web_tool(query: str, num_results: int = 5, score_threshold: float = 0.7):
    return web_search(query, num_results, score_threshold)
 
@tool("rag_search")
def rag_tool(search_params: list[dict], query: str, model: str) -> str:
    nvidia_rag = pytract_rag()
    return nvidia_rag.run_nvidia_text_generation_pipeline(search_params, query, model)
 
@tool("snowflake_search")
def snowflake_tool(query: str, filters: dict = {}):
    return "hi from snowflake"
 
# Define tool mapping dictionary
tool_str_to_func = {
    "web_search": web_tool,
    "rag_search": rag_tool,
    "snowflake_search": snowflake_tool
}
 
# Scratchpad function to track intermediate steps
def create_scratchpad(intermediate_steps: list[AgentAction]):
    research_steps = []
    for i, action in enumerate(intermediate_steps):
        if action.log != "TBD":
            research_steps.append(
                f"Tool: {action.tool}, input: {action.tool_input}\n"
                f"Output: {action.log}"
            )
    return "\n---\n".join(research_steps)
 
# Central Node (Oracle)
llm = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0
)
 
def run_oracle(inputs):
    return llm(inputs)
 
def run_tool(inputs):
    tool_name = inputs["tool"]
    return tool_str_to_func[tool_name].invoke(input=inputs["tool_input"])
 
graph = StateGraph(AgentState)
 
graph.add_node("oracle", run_oracle)
graph.add_node("web_search", run_tool)
graph.add_node("rag_search", run_tool)
graph.add_node("snowflake_search", run_tool)
graph.add_node("generate_report", run_tool)
 
graph.set_entry_point("oracle")
 
# Create edges between nodes
graph.add_edge("web_search", "oracle")
graph.add_edge("rag_search", "oracle")
graph.add_edge("snowflake_search", "oracle")
graph.add_edge("generate_report", END)
 
def generate_report(context):
    return f"Research Report:\n\nHistorical Performance:\n{context['rag_search']}\n\nFinancial Metrics:\n{context['snowflake_search']}\n\nIndustry Insights:\n{context['web_search']}"
 
graph.add_node("generate_report", generate_report)

ModuleNotFoundError: No module named 'langchain_openai'